# Notebook of Tremblay and Roach (n.d.)

**Abstract**:
*Interactions between ocean surface waves and sea ice dictate the width of the marginal ice zone, where new ice formation and increased sea ice melt are present in the winter and summer (respectively). Existing sea ice wave fracture models predict fracture when one of two limits is reached: (i) a maximum strain failure criterion assuming that the ice is a perfectly flexible plate that follows the ocean surface, and (ii) a maximum stress failure criterion assuming that the ice is a perfectly rigid plate that does not deform under the action of buoyancy and gravity forces. The perfectly rigid sea ice plate model is valid for small wavelengths that have a short lever arm but systematically predicts fracture for long wavelengths irrespective of the amplitude because of the long lever arm. Conversely, the flexible plate model is valid for long wavelengths but systematically predicts fracture for short wavelengths because of the unrealistically large strain. In this work, we present a unified sea ice fracture model based on elastic beam theory for the bending of a sea ice plate (or floe) that is valid for all wavelengths. Our approach reduces to the rigid plate and fully flexible model for short and long incoming ocean wavelength limits, respectively. Results using a fully-developed ocean wave field show much smaller strain within the ice plate and a resulting floe size distribution after fracture with a higher mean and no floes in the smallest size categories. This distribution also aligns with correct ice thickness and Young's Modulus dependencies, matching observational evidence, and contrasts with results from perfectly rigid or flexible sea ice plate models.*


**Links**:
- https://github.com/ESCOMP/Icepack/commit/6fccdbfb3ed3a48b0ac7418661966ab7799af40c#diff-478c9c2e89a3bd4b9a5762a0bae193a44686980662b297f11023670946f841b0
- https://discovery.researcher.life/article/a-unified-sea-ice-fracture-model-for-climate-applications/68770f3c6466373986fc386261c57a71

In [1]:
import numpy as np

## Basics

## Proposed Model

First we define our model parameters and our domain for calculating the failure limits across an ice floe with length, $L$.

In [3]:
youngs_modulus = 10e9  # Young's Modulus for ice (Pa)
critical_strain = 3.e-5  # critical strain
dx = 0.5 # spatial step in metres

In [ ]:
L = 100 # length of ice floe in metres
Nx = int(np.round(L/dx + dx))  # number of spatial points
x = np.arange(-L/2, L/2, dx)  # coordinates around centre of the floe

Assuming a free-surface for simplicity, we use the open-water dispersion relation
$$ \lambda_i = \frac{g}{2 \pi f_i^2}. $$

Spectral amplitude coefficients are calculated using the same approach as *Horvat and Tziperman (2015)*:
$$ A_i = \sqrt{2 S(f) \Delta f}. $$

The phase of each amplitude is also given by
$$ \phi \sim \mathrm{U} (0, 2\pi)$$

In [ ]:
lambda_i = g / (2 * np.pi * f_i**2)
spectral_amplitude = np.sqrt(2 * S_f * delta_f)
phase = np.random.uniform(0, 2 * np.pi, size=spectral_amplitude)

Interia is calculated by
$$ I = \frac{1}{12}\bar{h} ^3$$
for mean ice thickness, $\bar{h}$.

The characteristic (or natural) floe size (referred to as $\Lambda$ in the code) is subsequently defined as (Squire, 2007:
$$ L_c = \left( \frac{F}{\rho_w g} \right)^{1/4}$$
for a Young's modulus, $F$, and water density, $\rho_w$.

In [ ]:
characteristic_length = (youngs_modulus / (rho_w * g))**0.25

They then define a floe length, scaled by the characteristic flexural length
$$ \gamma = \frac{L}{2\sqrt{2} L_c}$$
which represents how long or short the floe is in relation to its flexural response length.

In [ ]:
gamma = L/(c2*SQRT(c2)*Lambda)

A characteristic wavelength is also defined as
$$ \langle l_i \rangle = \frac{\lambda_i}{2 \pi} $$

to compute an effective amplitude
$$ A_{\text{eff}} = A_i  \frac{\langle l_i \rangle^4}{L_c^4 + \langle l_i \rangle^4}, $$

and a non-dimensionalised coordinate
$$ x_p = \frac{x}{\sqrt{2} L_c}$$

In [ ]:
langi = lamdai/(c2*pi)

AAmi = spec_coeff*langi**4/(Lambda**4 + langi**4)
xp = x/(SQRT(c2)*Lambda)


\begin{aligned}
m &= \frac{6}{L^2} \sum_i \frac{A_i \, \lambda_i}{\pi} \, \sin(\phi_i)
\left[ -\cos\left(\frac{\pi L}{\lambda_i}\right) + \frac{\lambda_i}{\pi L} \, \sin\left(\frac{\pi L}{\lambda_i}\right) \right] \\[2mm]
b &= \frac{c_1}{L} \sum_i \frac{A_i \, \lambda_i}{\pi} \, \sin\left(\frac{\pi L}{\lambda_i}\right) \cos(\phi_i) - \frac{\rho_i}{\rho_w} h \\[1mm]
b’ &= -b - \frac{\rho_i}{\rho_w} h \\[1mm]
a’ &= -m
\end{aligned}

Then if $L < $ 300 m, the homogenous solution is computed for $A \mathrm{y_p} = b$, given
$$
\mathbf{A} =
\begin{bmatrix}
e^{-\gamma} \sin\gamma & e^{-\gamma} \cos\gamma & -e^{\gamma} \sin\gamma & -e^{\gamma} \cos\gamma \\
	-e^{\gamma} \sin\gamma & e^{\gamma} \cos\gamma & e^{-\gamma} \sin\gamma & - e^{-\gamma} \cos\gamma \\
\sin\gamma \cosh\gamma + \cos\gamma \sinh\gamma & -\cos\gamma \sinh\gamma + \sin\gamma \cosh\gamma & \sin\gamma \cosh\gamma + \cos\gamma \sinh\gamma & -\sin\gamma \cosh\gamma + \cos\gamma \sinh\gamma \\
\gamma \sin\gamma \sinh\gamma + \gamma \cos\gamma \cosh\gamma - \sin\gamma \cosh\gamma & \gamma \sin\gamma \sinh\gamma - \gamma \cos\gamma \cosh\gamma + \cos\gamma \sinh\gamma & -\gamma \cos\gamma \cosh\gamma - \gamma \sin\gamma \sinh\gamma + \sin\gamma \cosh\gamma & -\gamma \cos\gamma \cosh\gamma + \gamma \sin\gamma \sinh\gamma + \cos\gamma \sinh\gamma
\end{bmatrix}
$$

under the forcing
$$
\mathbf{b} =
\begin{bmatrix}
L_c^2 \sum_i \frac{A_{\text{eff}}}{\langle l_i \rangle^2} \cos\left(\frac{L}{2\langle l_i \rangle} + \phi_i\right) \\[1em]
L_c^2 \sum_i \frac{A_{\text{eff}}}{\langle l_i \rangle^2} \cos\left(\frac{L}{2\langle l_i \rangle} - \phi_i\right) \\[0.5em]
	- \frac{\sqrt{2}}{2 L_c} \left[ \sum_i A_{\text{eff}} \langle l_i \rangle \left( \sin\left(\frac{L}{2\langle l_i \rangle} - \phi_i\right) + \sin\left(\frac{L}{2 \langle l_i \rangle} + \phi_i\right) \right) + b’ L \right] \\[0.5em]
	- \frac{1}{2 L_c^2} \left[ \sum_i A_{\text{eff}} \langle l_i \rangle \left( \frac{L}{2}\left(\sin\left(\frac{L}{2 L_i} - \phi_i\right) - \sin\left(\frac{L}{2 L_i} + \phi_i\right)\right) + \langle l_i \rangle \left( \cos\left(\frac{L}{2\langle l_i \rangle} - \phi_i\right) - \cos\left(\frac{L}{2\langle l_i \rangle} + \phi_i\right) \right) \right) + \frac{a’ L^3}{12} \right]
\end{bmatrix}
$$

And then a homogenous solution is found
$$
    y_H = e^{x_p}(y_{P,1} \cos(x_p)+y_{P,2}*\sin(x_p)) + e^{-x_p}(y_{P,3}\cos(x_p)+y_{P,4}\sin(x_p))
$$

$$
    y_{ppH} = \frac{1}{L_c^2}(e^{x_p} (-y_{P,1}\sin(x_p)+y_{P,2}\cos(xp)) - e^{-x_p}(-y_{P,3}\sin(x_p)+y_{P,4}\cos(x_p)))
$$
           
